In [1]:
import numpy as np
from numpy.lib.function_base import average
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler

# 种群规模
pop_size = 50
# 种群范围
weight_range = [[0.5, 1.1], [0.6, 1.3], [0.1, 0.7], [0.5, 0.8],
                [1.5, 2.5], [2.0, 3.0], [0.1, 0.5], [2.0, 3.0],
                [1.5, 2.5], [1.5, 2.5], [1.5, 2.3], [2.5, 4.0]]
# 交叉率 一般在0.6~0.9之间
cross_rate = 0.8
# 变异率 一般小于0.1
mutation_rate = 0.05
# 种群迭代次数
n_genes = 200
# 最佳保留比例
save_best_rate= 0.1

# 编码值转权重值
def code2weight(code_li):
    return np.array([abs(code) / np.sum(np.abs(code_li)) for code in code_li])


#  适应度计算
def cal_fitness(x_train, y_train, code_li):
    weight_li = np.array([code if code_li.index(code) not in [0, 4, 7, 9, 10,11] else 
                 -1*code for code in code_li])
    weight_li = weight_li.reshape(-1, 1)
    y_fit = x_train.dot(weight_li)
    y_fit = y_fit.T
    return np.mean(np.sqrt((y_fit - y_train) ** 2))

# 初始化种群
def init_population(population_size, weight_range):
    #二维列表，包含染色体和基因
    population = []
    for i in range(population_size):
        temporary = []
        for j in range(len(weight_range)):
            #在权重范围内产生随机数
            temporary.append(random.uniform(round(weight_range[j][0], 2), round(weight_range[j][1], 2)))
        population.append(temporary)
    return population
                             
# 生成适应度计算列表                             
def gen_fitness_res(x_train, y_train, pop_li):
    return [cal_fitness(x_train, y_train, pop) for pop in pop_li]

                             
# 最优保存策略                             
def save_best_individual(fitness_li, pop_li, best_n):
    fitness_sort_idx = np.argsort(fitness_li)
    fitness_choice_sort_idx = np.array(fitness_sort_idx[:best_n])
    fitness_sort_idx = fitness_sort_idx[best_n:]
    fitness_sort_idx = fitness_sort_idx.tolist()
    save_best_ind = pop_li[fitness_choice_sort_idx]
    return fitness_sort_idx, save_best_ind
            
                             
# 选择算子                             
def choice_operator(fitness_sort_idx, pop_li, best_n):
    choice_pop_li = []
    cnt = 0
    while cnt < len(fitness_sort_idx):
        idx = random.choice(fitness_sort_idx)
        print(idx)
        probability = 1 - (fitness_sort_idx.index(idx)+ best_n) * 0.02
        ret = random.random()
        if ret <= probability:
            choice_pop_li.append(pop_li[idx])
            cnt += 1
    return choice_pop_li


# 交叉算子
def cross_operator(pop_li, alfa=0.8, numRangeList=[4] * 12, mu=0.7):
    """
        输入：
            population 种群
            alfa 交叉概率
            numRangeList 决策变量上限
            mu是一个（0,1）的随机数
    """
    population = np.array(pop_li)
    N = population.shape[0]
    V = population.shape[1]
    populationList = range(N)

    for _ in range(N):
        r = random.random()

        if r < alfa:
            p1, p2 = random.sample(populationList, 2)
            beta = np.array([0] * V)
            randList = np.random.random(V)
            
            for j in range(V):
                if randList.any() <= 0.5:
                    beta[j] = (2.0 * randList[j]) ** (1.0 / (mu + 1))
                else:
                    beta[j] = (1.0 / (2.0 * (1 - randList[j]))) ** (1.0 / (mu + 1))

                # 随机选取两个个体
                old_p1 = population[p1,]
                old_p2 = population[p2,]
                
                # 交叉
                new_p1 = 0.5 * ((1 + beta) * old_p1 + (1 - beta) * old_p2)
                new_p2 = 0.5 * ((1 - beta) * old_p1 + (1 + beta) * old_p2)

                # 上下界判断
                new_p1 = np.max(np.vstack((new_p1, np.array([0] * V))), 0)
                new_p1 = np.min(np.vstack((new_p1, numRangeList)), 0)

                new_p2 = np.max(np.vstack((new_p2, np.array([0] * V))), 0)
                new_p2 = np.min(np.vstack((new_p2, numRangeList)), 0)
                
                # 将交叉后的个体返回给种群
                population[p1,] = new_p1
                population[p2,] = new_p2
    pop_li = population.tolist()
    return pop_li


# 变异算子
def variation_operator(pop_li, mutation_rate):
    for i in range(len(pop_li)):
        for j in range(len(pop_li[i])):
            p = random.random()
            if p < mutation_rate:
                pop_li[i][j] = abs(np.random.normal(pop_li[i][j], pop_li[i][j] ** 2))
    return pop_li

    
# 数据准备                             
def gen_dataset():
    dataset =  read_csv("D:/work/kyotta/08_专利论文/22年风能展论文/boston.csv", ).values
    # 划分训练集和测试集（+数据标准化）
    X = np.array(dataset[:,0:12])
    Y = np.array(dataset[:,12])
    stand = StandardScaler()
    X_std = stand.fit_transform(X)
    x_train, x_test, y_train, y_test = train_test_split(X_std, Y, test_size=0.3)
    return x_train, y_train


def process_main():
    # 生成训练数据集
    x_train, y_train = gen_dataset()
    # 初始化种群
    pop_li = init_population(pop_size, weight_range)
    for i in range(n_genes):
        # 适应度计算
        fitness_li = gen_fitness_res(x_train, y_train, pop_li)
        pop_li = np.array(pop_li)
        print(pop_li)
        # 最优保存策略
        best_n = int(pop_size * save_best_rate)
        print(best_n)
        fitness_sort_idx, best_ind = save_best_individual(fitness_li, pop_li, best_n)
        # 选择
        pop_li = choice_operator(fitness_sort_idx, pop_li, best_n)
        # 交叉
        pop_li = cross_operator(pop_li)
        # 变异
        pop_li = variation_operator(pop_li, mutation_rate)
        # 合并最优的个体，形成新种群
        pop_li = np.vstack((pop_li, best_ind))
        pop_li = pop_li.tolist()
    print(best_ind, fitness_li)     


process_main()


NameError: name 'read_csv' is not defined